In [2]:
import pandas as pd
import ast


def car_overview():
    new_car_overview_df = pd.DataFrame(main_df['new_car_overview'].apply(ast.literal_eval).tolist())
    rows_list = list()
    for row in new_car_overview_df['top']:
        value_dict = {item['key']: item['value'] for item in row}
        rows_list.append(value_dict)
    new_car_overview_df = pd.DataFrame(rows_list)
    return new_car_overview_df


def car_feature():
    new_car_feature_df = pd.DataFrame(main_df['new_car_feature'].apply(ast.literal_eval).tolist())
    unique_top_features = set()
    for row in new_car_feature_df['top']:
        for item in row:
            unique_top_features.add(item['value'])
    unique_top_features = list(unique_top_features)
    rows_list_top = list()
    for row in new_car_feature_df['top']:
        row_dict = dict.fromkeys(unique_top_features, 0)
        for item in row:
            row_dict[item['value']] = 1
        rows_list_top.append(row_dict)

    new_car_feature_top_df = pd.DataFrame(rows_list_top)

    unique_features = set()
    for row in new_car_feature_df['data']:
        for item in row:
            items_list = item['list']
            for item in items_list:
                unique_features.add(item['value'])
    unique_features = list(unique_features)
    rows_list = list()
    for row in new_car_feature_df['data']:
        row_dict = dict.fromkeys(unique_features, 0)
        for item in row:
            items_list = item['list']
            for feature in items_list:
                row_dict[feature['value']] = 1
        rows_list.append(row_dict)
    new_car_feature_data_df = pd.DataFrame(rows_list)

    new_car_feature_df = pd.concat([new_car_feature_top_df, new_car_feature_data_df], axis=1)
    return new_car_feature_df

def car_specs():
    new_car_specs_df = pd.DataFrame(main_df['new_car_specs'].apply(ast.literal_eval).tolist())
    rows_list = list()
    for row in new_car_specs_df['top']:
        value_dict = {item['key']: item['value'] for item in row}
        rows_list.append(value_dict)
    new_car_specs_top_df = pd.DataFrame(rows_list)

    engine_list = list()
    dimension_list = list()
    misc_list = list()
    for row in new_car_specs_df['data']:
        if len(row) == 3:
            list1 = row[0].get('list', {})
            value_dict = {dicts['key']: dicts['value'] for dicts in list1}
            engine_list.append(value_dict)
            list2 = row[1].get('list', {})
            value_dict = {dicts['key']: dicts['value'] for dicts in list2}
            dimension_list.append(value_dict)  
            list3 = row[2].get('list', {})
            value_dict = {dicts['key']: dicts['value'] for dicts in list3}
            misc_list.append(value_dict)
        elif row[0].get('heading', {}) == 'Miscellaneous':
            engine_list.append({})
            dimension_list.append({})
            list3 = row[0].get('list', {})
            value_dict = {dicts['key']: dicts['value'] for dicts in list3}
            misc_list.append(value_dict)
        elif row[0].get('heading', {}) == 'Dimensions & Capacity':
            engine_list.append({})
            list2 = row[0].get('list', {})
            value_dict = {dicts['key']: dicts['value'] for dicts in list2}
            dimension_list.append(value_dict) 
            if row[1].get('heading', {}) == 'Miscellaneous':
                list3 = row[1].get('list', {})
                value_dict = {dicts['key']: dicts['value'] for dicts in list3}
                misc_list.append(value_dict)
            else:
                misc_list.append({})
        else:
            engine_list.append({})
            dimension_list.append({})
            misc_list.append({})

    new_car_specs_data_e_df = pd.DataFrame(engine_list)
    new_car_specs_data_d_df = pd.DataFrame(dimension_list)
    new_car_specs_data_m_df = pd.DataFrame(misc_list)

    new_car_specs_df = pd.concat([new_car_specs_top_df, new_car_specs_data_e_df, new_car_specs_data_d_df, new_car_specs_data_m_df], axis=1)
    
    return new_car_specs_df

list_of_cities = ['bangalore', 'chennai', 'delhi', 'hyderabad', 'jaipur', 'kolkata']

all_cities_df_list = list()

for i in list_of_cities:
    print(f'{i}_cars.xlsx')
    main_df = pd.read_excel(f'{i}_cars.xlsx')
    main_df['City'] = i
    all_cities_df_list.append(main_df)

main_df = pd.concat(all_cities_df_list, ignore_index=True)
    
new_car_detail_df = pd.DataFrame(main_df['new_car_detail'].apply(ast.literal_eval).tolist())
new_car_overview_df = car_overview()
new_car_feature_df = car_feature()
new_car_specs_df = car_specs()
raw_df = pd.concat([new_car_detail_df.reset_index(drop=True), 
                        new_car_overview_df.reset_index(drop=True), 
                        new_car_specs_df.reset_index(drop=True),
                        main_df['City'].reset_index(drop=True), 
                        main_df['car_links'].reset_index(drop=True),
                        new_car_feature_df.reset_index(drop=True)
                        ], axis=1)
        
raw_df.to_csv('Raw_Data.csv')

bangalore_cars.xlsx
chennai_cars.xlsx
delhi_cars.xlsx
hyderabad_cars.xlsx
jaipur_cars.xlsx
kolkata_cars.xlsx
